In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
%store -r student_courses
%store -r student_majors
%store -r stem_paths
%store -r jacobs
%store -r jacobs_ethnicities
%store -r bcdi
%store -r waivers
%store -r beeps

In [3]:
jacobs_ethnicities

,year.name,ppsk,undergrad.status,course.department,course.subject,course.number,offering.type,section.number,ccn,room.share,course.title,grade.subtype,grade.type,grade.name,grade.points,student.credit.hours,Ethnicity,gender
8,2015 Spring,1030081,Undergraduate,Engineering Science,Design Innovation,190,Primary,2,18406,18406.0,Spec Topics Des Inv,A,Letter Grade,A,4.0,2.0,Asian/Pacific Islander,Female
10,2015 Spring,1031223,Undergraduate,Engineering Science,Design Innovation,190,Primary,2,18406,18406.0,Spec Topics Des Inv,A,Letter Grade,A,4.0,2.0,Asian/Pacific Islander,Female
13,2015 Spring,1032550,Undergraduate,Engineering Science,Design Innovation,190,Primary,2,18406,18406.0,Spec Topics Des Inv,A,Letter Grade,A,4.0,2.0,Asian/Pacific Islander,Male
14,2015 Spring,1032733,Undergraduate,Engineering Science,Design Innovation,190,Primary,1,18403,18403.0,Spec Topics Des Inv,A,Letter Grade,A,4.0,2.0,Asian/Pacific Islander,Male
17,2015 Spring,1171276,Undergraduate,Engineering Science,Design Innovation,190,Primary,1,18403,18403.0,Spec Topics Des Inv,A,Letter Grade,A,4.0,2.0,Asian/Pacific Islander,Female
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1872,2019 Fall,2739108,Undergraduate,Design Innovation,Design Innovation,10,Primary,1,32998,32998.0,Discovering Design,A,Letter Grade,A-,3.7,2.0,White/Other,Female
1873,2019 Fall,2739284,Undergraduate,Design Innovation,Design Innovation,10,Primary,1,32998,32998.0,Discovering Design,A,Letter Grade,A,4.0,2.0,White/Other,Female
1876,2019 Fall,2741518,Undergraduate,Design Innovation,Design Innovation,22,Primary,1,28116,28116.0,Prototype & Fab,A,Letter Grade,A+,4.0,3.0,White/Other,Female
1877,2019 Fall,2741518,Undergraduate,Design Innovation,Design Innovation,190,Primary,1,15094,1721.0,Spec Topics Des Inv,A,Letter Grade,A,4.0,3.0,White/Other,Female


In [4]:
cleaned_jacobs = jacobs.loc[jacobs['ppsk'] != 'MISSING']
cleaned_jacobs['ppsk'] = cleaned_jacobs.loc[:, 'ppsk'].astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
urms = student_courses[['ppsk']].merge(stem_paths[['ppsk', 'ethnic.l1', 'gender']], on='ppsk', how='inner')
urms = urms[urms['ethnic.l1'] == 'Underrepresented Minority']
urms.drop_duplicates(inplace=True)

In [6]:
urms

,ppsk,ethnic.l1,gender
1630,787652,Underrepresented Minority,Male
1735,791882,Underrepresented Minority,Female
2169,813809,Underrepresented Minority,Male
2811,609891,Underrepresented Minority,Male
2997,656505,Underrepresented Minority,Female
...,...,...,...
3761375,2768138,Underrepresented Minority,Female
3761396,2769675,Underrepresented Minority,Male
3761586,1487118,Underrepresented Minority,Female
3761774,2428229,Underrepresented Minority,Male


In [7]:
urms.merge(student_majors[['ppsk', 'college']], on='ppsk')
['Clg of Letters & Science']

['Clg of Letters & Science']

In [8]:
student_majors

,ppsk,undergrad.status,year.name,Count of Students,college,division,department,major,exam.units
65498,651215,U,2007 Fall,1,Clg of Letters & Science,L&S Math & Phys Sciences Div,Mathematics,Applied Mathematics,5.4
48603,626974,U,2007 Fall,1,Clg of Letters & Science,L&S Social Sciences Division,Interdisc Social Science Pgms,Development Studies,8.0
7565,329180,U,2007 Fall,1,Clg of Letters & Science,L&S Social Sciences Division,Interdisc Social Science Pgms,Cognitive Science,0.0
7568,329246,G,2007 Fall,1,Grad School of Education,Grad School of Education,Education,Education,0.0
48590,626956,G,2007 Fall,1,Grad School of Education,Grad School of Education,Education,Education,0.0
...,...,...,...,...,...,...,...,...,...
21431,3187659,U,2020 Spring,1,Clg of Letters & Science,L&S Administered Programs,L&S Computer Science,L&S Computer Science,NaN
21432,3187668,U,2020 Spring,1,Clg of Letters & Science,L&S Undeclared,L&S Undeclared,Letters & Sci Undeclared,NaN
21433,3187669,U,2020 Spring,1,Clg of Engineering,Clg of Engineering,Electrical Eng & Computer Sci,Electrical Eng & Comp Sci,NaN
21420,3187555,U,2020 Spring,1,Clg of Letters & Science,L&S Undergrad Studies Division,UG Interdisciplinary Studies,Media Studies,NaN
